In [3]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
download = drive.CreateFile({'id': '19lWsG9i0fYz7mqoULzvODKyIyQLuCTVS'})
download.GetContentFile('fruits-dataset.zip')

In [6]:
import zipfile
import io
data = zipfile.ZipFile('fruits-dataset.zip', 'r')
data.extractall()

os.rename('archive', 'fruits-dataset') # Just renaming the unzipped folder as it was zipped under the name archive

In [7]:
train_dir = 'fruits-dataset/train/train'
# validation_dir = 'fruits-dataset/validate/validate'
test_dir = 'fruits-dataset/test/test'

In [8]:
#Create the dataframe

import pandas as pd
import os

data = []
# print(os.listdir(train_dir))

for fruit_folder in os.listdir(train_dir):
  fruit_path = os.path.join(train_dir, fruit_folder)
  if os.path.isdir(fruit_path):
    for image_file in os.listdir(fruit_path):
      if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
          image_path = os.path.join(fruit_path, image_file)
          data.append({'image_url': image_path, 'fruit_name': fruit_folder})

df = pd.DataFrame(data)
print(df.head())


                                           image_url  fruit_name
0  fruits-dataset/train/train/Pepper Red/Pepper R...  Pepper Red
1  fruits-dataset/train/train/Pepper Red/Pepper R...  Pepper Red
2  fruits-dataset/train/train/Pepper Red/Pepper R...  Pepper Red
3  fruits-dataset/train/train/Pepper Red/Pepper R...  Pepper Red
4  fruits-dataset/train/train/Pepper Red/Pepper R...  Pepper Red


In [9]:
batch_size=64 # Change to 16 or 8
image_size=100
#################################### Data Augmentation ###################################
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=20,
                                 height_shift_range=0.2,
                                 width_shift_range=0.2,
                                 shear_range=0.2,
                                 horizontal_flip=True,
                                 zoom_range=0.2,
                                fill_mode='nearest')


train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(image_size,image_size),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

Found 16854 images belonging to 33 classes.


In [17]:
from tensorflow.keras import layers, optimizers
from tensorflow.keras import models, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import load_model

model_path = "fruit_classification_model.keras"

if os.path.exists(model_path):
    print("Loading existing model...")
    model = load_model(model_path)
else:
    print("Model not found. Creating a new model...")
    
    model = models.Sequential()
    
    model.add(Input(shape=(image_size, image_size, 3)))
    model.add(layers.Conv2D(64, (3, 3),padding='valid'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3),padding='valid'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3),padding='valid'))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100))
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(33))
    model.add(layers.Activation('softmax'))
    
    
    
    checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5.keras', monitor='val_loss', verbose = 1, save_best_only=True)
    early = EarlyStopping(monitor='accuracy', min_delta=0, patience=10, verbose=1, mode='auto') #val_acc
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    
    history=model.fit(train_generator,
                      steps_per_epoch=int(train_generator.samples/train_generator.batch_size),
                      epochs=20,
                      verbose=1,
                      callbacks=[checkpointer, early])
    
    # Save the model for future use
    model.save(model_path)
    print("New model created and saved.")


Loading existing model...


In [16]:
# Save the full model
model.save("fruit_classification_model.keras")

In [11]:
class_indices = train_generator.class_indices  # Dictionary mapping class names to indices
index_to_class = {v: k for k, v in class_indices.items()}  # Reverse mapping
index_to_class

{0: 'Apple Braeburn',
 1: 'Apple Granny Smith',
 2: 'Apricot',
 3: 'Avocado',
 4: 'Banana',
 5: 'Blueberry',
 6: 'Cactus fruit',
 7: 'Cantaloupe',
 8: 'Cherry',
 9: 'Clementine',
 10: 'Corn',
 11: 'Cucumber Ripe',
 12: 'Grape Blue',
 13: 'Kiwi',
 14: 'Lemon',
 15: 'Limes',
 16: 'Mango',
 17: 'Onion White',
 18: 'Orange',
 19: 'Papaya',
 20: 'Passion Fruit',
 21: 'Peach',
 22: 'Pear',
 23: 'Pepper Green',
 24: 'Pepper Red',
 25: 'Pineapple',
 26: 'Plum',
 27: 'Pomegranate',
 28: 'Potato Red',
 29: 'Raspberry',
 30: 'Strawberry',
 31: 'Tomato',
 32: 'Watermelon'}

In [12]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_fruit(image_path, model, index_to_class, image_size=100):
    img = image.load_img(image_path, target_size=(image_size, image_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class_name = index_to_class[predicted_class_index]

    return predicted_class_name

# Example usage:
# image_path = "path_to_test_image.jpg"
# print("Predicted fruit:", predict_fruit(image_path, model, index_to_class))

In [18]:
import os

# print(os.listdir(test_dir))
max_count = 100
count = 0
results = []
for fruit_file in os.listdir(test_dir):
    if count > max_count:
        break
    if fruit_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        count += 1
        image_path = os.path.join(test_dir, fruit_file)
        # image_path = os.path.join(fruit_folder, image_file)
        predicted_class = predict_fruit(image_path, model, index_to_class)
        tempResult = {"image": fruit_file, "actual_class": image_path, "predicted_class": predicted_class}
        print(tempResult)
        results.append(tempResult)

# Convert results to a DataFrame for analysis
df_results = pd.DataFrame(results)
print(df_results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
{'image': '3067.jpg', 'actual_class': 'fruits-dataset/test/test/3067.jpg', 'predicted_class': 'Passion Fruit'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
{'image': '4237.jpg', 'actual_class': 'fruits-dataset/test/test/4237.jpg', 'predicted_class': 'Tomato'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
{'image': '2140.jpg', 'actual_class': 'fruits-dataset/test/test/2140.jpg', 'predicted_class': 'Peach'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
{'image': '2052.jpg', 'actual_class': 'fruits-dataset/test/test/2052.jpg', 'predicted_class': 'Apple Granny Smith'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
{'image': '2832.jpg', 'actual_class': 'fruits-dataset/test/test/2832.jpg', 'predicted_class': 'Pomegranate'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
{'image': '4975.jpg', 'actual_class': 'fruits-dataset/test/test/4975.jpg', 'predicted_class': 'Peach'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
{'image': '3958.jpg', 'actual_class': 'fruits-dataset/test/test/3958.jpg', 'predicted_c